In [1]:
import pandas as pd
import requests
import json

KOBERT_URL = 'http://3.35.8.82:5000/kobert?text='
BERT_URL = 'http://3.35.8.82:5000/bert?text='

In [2]:
movie = pd.read_csv('movie_datasets.csv')
movie2 = pd.read_csv('af_kmdb.csv')
book = pd.read_csv('book_datasets.csv')
book2 = pd.read_csv('naver_books.csv')
music = pd.read_csv('music_datasets.csv')
music2 = pd.read_csv('music_datasets_4000.csv')


In [3]:
book['title'] = book['title'].apply(lambda x:x.split('</b>')[0])

In [4]:
book.head(20)

title                                               link  \
0             82년생 김지영  http://book.naver.com/bookdb/book_detail.php?b...   
1          나미야 잡화점의 기적  http://book.naver.com/bookdb/book_detail.php?b...   
2               언어의 온도  http://book.naver.com/bookdb/book_detail.php?b...   
3          돌이킬 수 없는 약속  http://book.naver.com/bookdb/book_detail.php?b...   
4                  아몬드  http://book.naver.com/bookdb/book_detail.php?b...   
5   죽고 싶지만 떡볶이는 먹고 싶어2  http://book.naver.com/bookdb/book_detail.php?b...   
6         나는 나로 살기로 했다  http://book.naver.com/bookdb/book_detail.php?b...   
7                채식주의자  http://book.naver.com/bookdb/book_detail.php?b...   
8               당신이 옳다  http://book.naver.com/bookdb/book_detail.php?b...   
9                 사피엔스  http://book.naver.com/bookdb/book_detail.php?b...   
10              열두 발자국  http://book.naver.com/bookdb/book_detail.php?b...   
11            공부머리 독서법  http://book.naver.com/bookdb/book_detail.php?b...   
12      하마터면 열심히 살 뻔했다  http://book.naver.com/bookdb/book_detail.php?b...   
13              여행의 이유  http://book.naver.com/bookdb/book_detail.php?b...   
14          걷는 사람, 하정우  http://book.naver.com/bookdb/book_detail.php?b...   
15              아가씨와 밤  http://book.naver.com/bookdb/book_detail.php?b...   
16             오직 두 사람  http://book.naver.com/bookdb/book_detail.php?b...   
17       고요할수록 밝아지는 것들  http://book.naver.com/bookdb/book_detail.php?b...   
18              소년이 온다  http://book.naver.com/bookdb/book_detail.php?b...   
19             미움받을 용기  http://book.naver.com/bookdb/book_detail.php?b...   

                                                image            author  \
0   https://bookthumb-phinf.pstatic.net/cover/112/...               조남주   
1   https://bookthumb-phinf.pstatic.net/cover/071/...          히가시노 게이고   
2   https://bookthumb-phinf.pstatic.net/cover/109/...               이기주   
3   https://bookthumb-phinf.pstatic.net/cover/116/...           야쿠마루 가쿠   
4   https://bookthumb-phinf.pstatic.net/cover/118/...               손원평   
5   https://bookthumb-phinf.pstatic.net/cover/148/...               백세희   
6   https://bookthumb-phinf.pstatic.net/cover/214/...               김수현   
7   https://bookthumb-phinf.pstatic.net/cover/222/...                한강   
8   https://bookthumb-phinf.pstatic.net/cover/140/...               정혜신   
9   https://bookthumb-phinf.pstatic.net/cover/175/...  유발 하라리|다비드 반데르묄렝   
10  https://bookthumb-phinf.pstatic.net/cover/137/...               정재승   
11  https://bookthumb-phinf.pstatic.net/cover/135/...               최승필   
12  https://bookthumb-phinf.pstatic.net/cover/135/...                하완   
13  https://bookthumb-phinf.pstatic.net/cover/147/...               김영하   
14  https://bookthumb-phinf.pstatic.net/cover/142/...               하정우   
15  https://bookthumb-phinf.pstatic.net/cover/142/...             기욤 뮈소   
16  https://bookthumb-phinf.pstatic.net/cover/120/...               김영하   
17  https://bookthumb-phinf.pstatic.net/cover/142/...                혜민   
18  https://bookthumb-phinf.pstatic.net/cover/076/...                한강   
19  https://bookthumb-phinf.pstatic.net/cover/104/...   기시미 이치로|고가 후미타케   

    price  discount publisher   pubdate                      isbn  \
0   13000   11700.0       민음사  20161014  8937473135 9788937473135   
1   14800   13320.0      현대문학  20121219  8972756199 9788972756194   
2   13800   12420.0       말글터  20160819  1195522126 9791195522125   
3   15000   13500.0      북플라자  20170202  8998274795 9788998274795   
4   12000   10800.0        창비  20170331  8936434268 9788936434267   
5   13800   12420.0         흔  20190503  1196394571 9791196394578   
6   16000   14400.0    클레이하우스  20220204  1197377158 9791197377150   
7   15000   13500.0        창비  20220328  8936434594 9788936434595   
8   15800   14220.0     해냄출판사  20181010  8965746663 9788965746669   
9   17800   16020.0       김영사  20201123  8934991321 9788934991328   
10  16800   15120.0      어크로스  20180702  116056051X 979116056051

In [5]:
music['emotions'][0]

"{'공포': 0.0702, '놀람': 0.0528, '분노': 0.1024, '슬픔': 0.1518, '중립': 0.7047, '행복': 0.0297, '혐오': 0.0374}"

In [6]:
def getEmotionsFromAPI(model='kobert', text=''):
    url = KOBERT_URL if model=='kobert' else BERT_URL
    res = requests.get(url + text).text
    return json.loads(res)

def getDistance(user_emo, media_emo):
    total = 0
    for key in user_emo.keys():
        n1 = user_emo[key]
        n2 = media_emo[key]
        total += (n1-n2)**2
    return total

In [7]:
from tqdm.notebook import tqdm
def getSeries(series, model='kobert'):
    tmp = []
    for row in tqdm(series):
        tmp.append(getEmotionsFromAPI(model=model, text=row))
    print('len:', len(tmp))
    return pd.Series(tmp)
# book['emotions'] = book['description'].apply(lambda x:getEmotionsFromAPI('kobert', x))
    

In [8]:
book['emotions'] = getSeries(book['description'], 'kobert')
music['emotions'] = getSeries(music['new_lyrics'], 'bert')
movie['emotions'] = getSeries(movie['content'], 'kobert')

  0%|          | 0/1021 [00:00<?, ?it/s]

len: 1021


  0%|          | 0/1306 [00:00<?, ?it/s]

len: 1306


  0%|          | 0/15545 [00:00<?, ?it/s]

len: 15545


In [9]:
movie.to_csv('movie_datasets.csv', index=False)
book.to_csv('book_datasets.csv', index=False)
music.to_csv('music_datasets.csv', index=False)

In [ ]:
print(movie['title'][0])
row = movie['content'][0]
print(row)
print(getEmotionsFromAPI('kobert', row))

에브리씽 유브 갓 하우 두 유 노우?
소프트볼,국가대표,탈락,투수,중역,회계,부정,누명
{'공포': 0.0029, '놀람': 0.0111, '분노': 0.1089, '슬픔': 0.04, '중립': 0.0386, '행복': 0.003, '혐오': 0.7956}
